# 종합실습문제1. 로그 데이터 분석

다음 데이터는 깃허브 서비스에서 요일 별 사용자들의 푸시 횟수로, 요일별로 푸시 횟수가 다르다고 한다. 분산분석을 수행하여 검증하시오.

- 데이터파일 : log_push.csv

### 필요 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

%precision 5

'%.5f'

In [2]:
plt.rcParams['font.family'] = 'Malgun Gothic'

#### 분산분석을 위한 모듈들 임포트

In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import MultiComparison

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
!pip install pingouin

In [ ]:
import pingouin as pg

### 데이터 수집

In [ ]:
df = pd.read_csv('data/log_push.csv')

### 데이터 기초 정보 파악

: 데이터 일부, 변수, 관측값수, 결측치 여부, 변수의 데이터타입, 기술통계량 등을 확인

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### 데이터 가공

#### log_date컬럼을 이용하여 요일 컬럼(day_of_week) 생성

- log_date를 날짜형식으로 변경

In [ ]:
df.log_date = pd.to_datetime(df.log_date, format='%y-%m-%d')
df.head()

In [ ]:
df.info()

- log_date를 이용하여 요일 컬럼(day_of_week) 생성

In [ ]:
df['day_of_week'] = df.log_date.dt.day_of_week
# 0 : Monday, 6: Sunday

In [ ]:
df.head()

- 요일 컬럼의 데이터를 범주형(Category)으로 변경

In [ ]:
df['day'] = pd.cut(df.day_of_week, bins=[-1,0,1,2,3,4,5,6], labels=['Mon','Tue','Wed','Thr','Fri','Sat','Sun'])
df.head()

In [ ]:
df.info()

In [ ]:
df['day2'] = df.log_date.dt.day_name()
df.info()

### 데이터 탐색 및 시각화

#### 요일별 빈도수

In [ ]:
df.day.value_counts()

#### 요일별 push수에 대한 기술통계량

In [ ]:
df.groupby(by='day')['push_count'].describe()

#### 요일별 push수에 대한 박스플롯

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(data=df, x='day', y='push_count')
plt.title('요일별 push counts')
plt.show()

#### push_count에 대한 요일별  히스토그램

- matplotlib을 이용하여 요일별 히스토그램을 한 윈도우에 겹쳐서 그리기

In [ ]:
x_min = df.push_count.min()
x_max = df.push_count.max()
bins = np.arange(x_min, x_max, step=(x_max-x_min)/20)

plt.figure(figsize=(10,5))
legs = []
for d, g in df.groupby(by='day'):
    g.push_count.hist(bins=bins, alpha=0.3)
    legs.append(d)
plt.legend(legs)
plt.xlabel('Push counts')
plt.ylabel('Day counts')
plt.show()

- seaborn을 이용하여 요일별 히스토그램 그리기

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=df, x='push_count', hue='day', alpha=0.3, kde=True)
plt.ylabel('Day Counts')
plt.show()

### 이상치 탐색 및 제거

In [ ]:
outlier_index = []

for d, g in df.groupby(by='day'):
    q = g.push_count.quantile([0.25, 0.75])
    iqr = q[0.75] - q[0.25]
    up_hinge = q[0.75] + iqr*1.5
    lw_hinge = q[0.25] - iqr*1.5
    outlier = g[(g.push_count > up_hinge) | (g.push_count < lw_hinge)]
    out_idx = list(outlier.index)
    print(f'{d}의 이상치는 {len(out_idx)}개, index={out_idx}')
    outlier_index = outlier_index + out_idx

df_no_outlier = df.drop(index=outlier_index)
df_no_outlier.head()

#### 이상치 제거 후 push_count의 요일별 박스플롯

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(data=df_no_outlier, x='day', y='push_count')
plt.title('요일별 push counts')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=df_no_outlier, x='push_count', hue='day', alpha=0.3, kde=True)
plt.ylabel('Day Counts')
plt.show()

## 일원분산분석

### 1. 가설 설정

### 2. 정규성 검정

- 원 데이터로 요일별 push_count의 정규성 검정

In [ ]:
df.push_count[df.day=='Thr']

In [ ]:
days = ['Mon','Tue','Wed','Thr','Fri','Sat','Sun']
print('정규성 검정')
for d in days:
    stat, p = stats.shapiro(df.push_count[df.day==d])
    print(f'{d} : shpiro stat={stat:.3f}, p-value={p:.3f}')

- 이상치 제거한 데이터로 요일별 push_count의 정규성 검정

In [ ]:
print('이상치 제거 후 정규성 검정')
for d in days:
    stat, p = stats.shapiro(df_no_outlier.push_count[df_no_outlier.day==d])
    print(f'{d} : shpiro stat={stat:.3f}, p-value={p:.3f}')

### 3. 등분산성 검정

- 이상치 제거한 데이터를 가지고 등분산성 검정

In [ ]:
mon = df_no_outlier.push_count[df_no_outlier.day == 'Mon']
tue = df_no_outlier.push_count[df_no_outlier.day == 'Tue']
wed = df_no_outlier.push_count[df_no_outlier.day == 'Wed']
thu = df_no_outlier.push_count[df_no_outlier.day == 'Thr']
fri = df_no_outlier.push_count[df_no_outlier.day == 'Fri']
sat = df_no_outlier.push_count[df_no_outlier.day == 'Sat']
sun = df_no_outlier.push_count[df_no_outlier.day == 'Sun']

stat, p = stats.levene(mon, tue, wed, thu, fri, sat, sun)
print(f'stat={stat:.3f}, p={p:.3f}')

### 4. 분산분석 수행
- 가정 검정 결과에 맞게 분석을 수행하시오.

In [ ]:
# 등분산성을 만족하지 않은 경우 -> Welch ANOVA로 분석
pg.welch_anova(data=df_no_outlier, dv='push_count', between='day')

=> Welch ANOVA 분석 결과 : p값에 매우 작아 그룹간 push count가 같다는 귀무가설을 기각하므로 요일별 push count가 다르다고 말할 수 있음

### 5. 사후검정 수행

- 어느 요일간에 차이가 있는가?

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

%precision 3

'%.3f'

In [ ]:
pd.read_excel

----------------------------------